# Logical Transduction

Agentics objects are capable of performing logical transduction between their states by using the provied LLMs. The logical transduction operator << can be applied between any two agentics of any type.

Transduction is done between a source and a target AG when connected by the << operator.

The follwing example transduces a Question into an Answer.

In [1]:
# ! uv pip install agentics-py

! uv pip install "git+https://github.com/IBM/agentics/@Colab"


import os
import sys

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/.env")
    from google.colab import drive

    drive.mount("/content/drive")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = input("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics0.0.6/Agentics/.venv
   Updating https://github.com/IBM/agentics/ (Colab)               
   Updating https://github.com/IBM/agentics/ (Colab)       
   Updating https://github.com/IBM/agentics/ (Colab)       
    Updated https://github.com/IBM/agentics/ (796dd9054e482a239b385753017d359cab8a85
Resolved 250 packages in 322ms                                       
Uninstalled 2 packages in 2ms
Installed 1 package in 1ms.post7.dev0+796dd90 (from git+http
 - agentics-py==0.0.4.post1.dev0+b2e926b (from file:///Users/gliozzo/Code/agentics0.0.6/Agentics)
 ~ agentics-py==0.0.10.post7.dev0+796dd90 (from git+https://github.com/IBM/agentics/@796dd9054e482a239b385753017d359cab8a8573)
In Colab: False
In Colab: False


In [2]:
from agentics import Agentics as AG
from pydantic import BaseModel
from typing import Optional
from agentics.core.llm_connections import get_llm_provider


## Define target and source types
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None

class Question(BaseModel):
    question :Optional[str] = None


## Instantiate the source AG with a question
source = AG(atype=Question,
            llm=get_llm_provider(), ## You can choose between "openai" (i.e. get_llm_provider("openai")), "watsonx", "gemini", "vllm_crewai"
                                    ## set verbose to true to see the internal agents log. This is optional.
            states=[Question(question="What is the capital of Italy?")])

## Instantiate the target AG with a target type. No instances are needed for zero shot transduction
target= AG(atype=Answer,  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
           verbose_agent=True)   ## set verbose to true to see the internal agents log

# Execute logical transduction by using the << operator between source and target AG
answer = await (target << source) ## Note that << operator is asyncronus and the results should be awaited

# Print the results of the transduction
print(answer.pretty_print()) ## Note that confidence is a float number, while other fields are strings

2025-09-05 09:25:17.605 | DEBUG    | agentics.core.llm_connections:<module>:121 - AGENTICS is connecting to the following LLM API providers:
2025-09-05 09:25:17.605 | DEBUG    | agentics.core.llm_connections:<module>:124 - 0 - WatsonX
2025-09-05 09:25:17.606 | DEBUG    | agentics.core.llm_connections:<module>:129 - 1 - Gemini
2025-09-05 09:25:17.606 | DEBUG    | agentics.core.llm_connections:<module>:133 - 2 - OpenAI
2025-09-05 09:25:17.606 | DEBUG    | agentics.core.llm_connections:<module>:135 - Please add API keys in .env file to add or disconnect providers.
2025-09-05 09:25:17.615 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-05 09:25:17.616 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
2025-09-05 09:25:17.617 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. U

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: aa37a26c-c7ba-4681-8d17-3da1568239cd                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│  Generate an object of the specified type from the following input. SOURCE:                                     │
│  {"question": "What is the capital of Italy?"}                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "answer": "Rome",                                                                                            │
│    "justification": "The capital of Italy is a well-known fact, and Rome is the correct answer.",               │
│    "confidence": 1.0                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 121a8f0f-6c02-4154-962b-a0e68a955fc9                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: aa37a26c-c7ba-4681-8d17-3da1568239cd                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "answer": "Rome",                                                                                            │
│    "justification": "The capital of Italy is a well-known fact, and Rome is the correct answer.",               │
│    "confidence": 1.0                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-09-05 09:25:19.494 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 1 states in 1.8753011226654053 seconds
2025-09-05 09:25:19.494 | DEBUG    | agentics.core.agentics:__lshift__:700 - 1 states processed in 0.09376505613327027 seconds average per state ...


Atype : <class '__main__.Answer'>
answer: Rome
justification: The capital of Italy is a well-known fact, and Rome is the correct
  answer.
confidence: 1.0




In agentics, lists of strings can be used as sources instead of AG. Those are provided as input for the transduction.

In [3]:
answer = await (AG(atype=Answer) << ["Where is Paris?"])
print(answer.pretty_print())

2025-09-05 09:25:19.499 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-05 09:25:19.499 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-05 09:25:21.030 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 1 states in 1.5306179523468018 seconds
2025-09-05 09:25:21.030 | DEBUG    | agentics.core.agentics:__lshift__:700 - 1 states processed in 0.07653089761734008 seconds average per state ...


Atype : <class '__main__.Answer'>
answer: Paris is the capital of France
justification: The question asks for the location of Paris, and based on general knowledge,
  Paris is the capital of France
confidence: 0.95




## Asyncronous Transduction

When the source AG has more than one state, the << operator perform asyncronous transfuction for each state to the target type. Transductions are executed in batches.

In [4]:
target = AG(atype=Answer,
            batch_size = 4, # This will process 2 questions per batch
            verbose_transduction = True, # Set to verbose to see transduction timings and other logs
            transduction_logs_path = "/tmp/answers.jsonl") # Optionally write longs of transductions on the specified path
questions= ["Where is Paris?",
            "Who is Alberto Sordi",
            "When will climate change be irreverible?",
            "Who is the best Jeopardy player?"]

answers = await (target << questions)

print(answer.pretty_print())

target.batch_size = 4  ## Processing 4 questions in parallel often results in
                                    ## lower execution time
                                    ## if supported by LLM backend

answers = await (target << questions)

2025-09-05 09:25:21.036 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy player?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-05 09:25:21.037 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-05 09:25:32.985 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 4 states in 11.947695970535278 seconds
2025-09-05 09:25:32.986 | DEBUG    | agentics.core.agentics:__lshift__:700 - 4 states processed in 2.9869239926338196 seconds average per state ...
2025-09-05 09:25:32.987 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopard

Atype : <class '__main__.Answer'>
answer: Paris is the capital of France
justification: The question asks for the location of Paris, and based on general knowledge,
  Paris is the capital of France
confidence: 0.95




2025-09-05 09:25:37.118 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 4 states in 4.130216121673584 seconds
2025-09-05 09:25:37.120 | DEBUG    | agentics.core.agentics:__lshift__:700 - 4 states processed in 1.032554030418396 seconds average per state ...


## Self Transduction

Self transduction is a method of AGs that enables async execution of transductions between slots of the same object.

In [5]:
from pydantic import BaseModel, Field
from typing import Optional
from pathlib import Path
## Define the Pydantic type
class Movie(BaseModel):
    movie_name: Optional[str] = None ## Note that fields name should match the column name in the input csv
    genre: Optional[str] = None
    description: Optional[str]=None
    tweet: Optional[str]=Field(None,description="Generate a Tweet to advertise the movie")
base = Path(CURRENT_PATH)
movies = AG.from_csv( base / "data/movies.csv", atype=Movie, max_rows=20) ## Load the input data from a csv file
movies.verbose_transduction = True
movies.llm=get_llm_provider("watsonx") ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"

movies_with_tweets = await movies.self_transduction(

    ["movie_name", "genre", "description"], ## source fields
    ["tweet"],                              ## target fields

    ## Note that instruction are only needed when the  relation between source and target type
    # is not innediately clear and need to be further specified or disambiguated.
    instructions="Generate a tweet to advertise the release of the input movie")

print(movies_with_tweets.pretty_print())

2025-09-05 09:25:37.128 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-05 09:25:37.130 | DEBUG    | agentics.core.llm_connections:get_llm_provider:42 - Using specified LLM provider: watsonx
2025-09-05 09:25:37.132 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
20 states will be transduced
2025-09-05 09:25:37.133 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-05 09:25:44.439 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 7.3056230545043945 seconds
2025-09-05 09:25:44.439 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.3652811527252197 seconds average per state ...


Atype : <class '__main__.Movie'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.
tweet: 'Get ready to be inspired! ''The Shawshank Redemption'', a gripping Drama &
  Crime film, is now out! Imprisoned for a crime he didn''t commit, Andy Dufresne''s
  story of hope, integrity & redemption will leave you moved. Don''t miss it! #TheShawshankRedemption
  #NewRelease'

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Cor

As an alternative, self transduction can be encoded using logical transduction algebra which uses the AG() notation to rebind the original data into AGs of the requested subtypes. Learn more about atype manipulation in agentics [here](link)

In [6]:

movies = AG.from_csv( base / "data/movies.csv", atype=Movie, max_rows=20)
tweets = await (AG(atype=movies("tweet").atype,
                   instructions="Generate a tweet to advertise the release of the input movie")
                << movies("movie_name","genre","description" ))
print(tweets.pretty_print())    ## Note that differently from self transduction the output tweets
                                ##has only the tweet field, whereas self transduction preserves
                                ## the original source data

2025-09-05 09:25:44.456 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-05 09:25:44.459 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
20 states will be transduced
2025-09-05 09:25:44.460 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-05 09:25:52.172 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 7.71193790435791 seconds
2025-09-05 09:25:52.173 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.38559689521789553 seconds average per state ...


Atype : <class 'agentics.core.agentics.tweet'>
tweet: 'Get ready to be inspired! ''The Shawshank Redemption'', a gripping Drama &
  Crime film, is now out! Imprisoned for a crime he didn''t commit, Andy Dufresne''s
  story of hope & redemption will leave you moved. Don''t miss it! #TheShawshankRedemption
  #NewRelease #MustWatch'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather'', a gripping Drama
  & Crime film, is out now! Spanning 1945 to 1955, this epic tale of the Italian-American
  Corleone crime family will keep you on the edge of your seat. Don''t miss it! #TheGodfather
  #Drama #Crime'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather Part II'' is now out!
  This gripping Drama, Crime film follows the Corleone crime family''s saga as they
  expand their empire. Don''t miss the epic story of Vito and Michael Corleone''s
  journey! #TheGodfatherPartII #Drama #Crime'

tweet: 'Get ready to witness a powerful true story! ''Schindler''s List'', a grippin

## Few Shots Transduction

Few shots examples can be provided for transduction by adding instances of the target instances in correspondance to their sources . Those will be used by the LLM to infer by analogy all the Null instances of the target type.


In [ ]:
movies = AG.from_csv(base / "data/movies.csv", max_rows=20)
## Note that obly the first 9 movies have categories
for i, movie in enumerate(movies): print(f"{i}: {movie.genre}")
## predicting new genre from given examples
all_genres=await (movies("genre")<<movies("movie_name",  "description"))
print(all_genres.pretty_print())
## few shots examples can also be used in self transfuction
movies_with_genre = await movies.self_transduction(["movie_name",  "description"],["genre"])
print(movies_with_genre.pretty_print())

2025-09-05 09:25:52.211 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate an object of the specified type from the following input.
20 states will be transduced
2025-09-05 09:25:52.212 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>


0: Drama, Crime
1: Drama, Crime
2: Drama, Crime
3: Drama, History, War
4: Drama
5: Animation, Family, Fantasy
6: Drama, Action, Crime, Thriller
7: Comedy, Drama, Romance
8: Fantasy, Drama, Crime
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 


2025-09-05 09:25:54.159 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 1.9466400146484375 seconds
2025-09-05 09:25:54.159 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.09733200073242188 seconds average per state ...
2025-09-05 09:25:54.161 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate an object of the specified type from the following input.
20 states will be transduced
2025-09-05 09:25:54.162 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>


Atype : <class 'agentics.core.agentics.genre'>
genre: Drama, Crime

genre: Drama, Crime

genre: Drama, Crime

genre: Drama, History, War

genre: Drama

genre: Animation, Family, Fantasy

genre: Drama, Action, Crime, Thriller

genre: Comedy, Drama, Romance

genre: Fantasy, Drama, Crime

genre: Drama, Comedy, Thriller

genre: Crime, Drama

genre: Fantasy, Romance

genre: Fantasy, Adventure, Drama

genre: Drama, Romance

genre: Western, Drama

genre: Action, Adventure, Drama

genre: Drama, Crime

genre: Adventure, Science Fiction

genre: Drama, War

genre: Drama, Comedy




2025-09-05 09:25:55.709 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 1.547492265701294 seconds
2025-09-05 09:25:55.710 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.0773746132850647 seconds average per state ...


Atype : <class 'agentics.core.utils.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Drama, Crime
description: In the